# How to Train YOLOv5 on Custom Objects

This tutorial is based on the [YOLOv5 repository](https://github.com/ultralytics/yolov5) by [Ultralytics](https://www.ultralytics.com/). This notebook shows training on **your own custom objects**. Many thanks to Ultralytics for putting this repository together - we hope that in combination with clean data management tools at Roboflow, this technologoy will become easily accessible to any developer wishing to use computer vision in their projects.

### Accompanying Blog Post

We recommend that you follow along in this notebook while reading the blog post on [how to train YOLOv5](https://blog.roboflow.ai/how-to-train-yolov5-on-a-custom-dataset/), concurrently.

### Steps Covered in this Tutorial

In this tutorial, we will walk through the steps required to train YOLOv5 on your custom objects. We use a [public blood cell detection dataset](https://public.roboflow.ai/object-detection/bccd), which is open source and free to use. You can also use this notebook on your own data.

To train our detector we take the following steps:

* Install YOLOv5 dependencies
* Download custom YOLOv5 object detection data
* Write our YOLOv5 Training configuration
* Run YOLOv5 training
* Evaluate YOLOv5 performance
* Visualize YOLOv5 training data
* Run YOLOv5 inference on test images
* Export saved YOLOv5 weights for future inference



### **About**

[Roboflow](https://roboflow.com) enables teams to deploy custom computer vision models quickly and accurately. Convert data from to annotation format, assess dataset health, preprocess, augment, and more. It's free for your first 1000 source images.

**Looking for a vision model available via API without hassle? Try Roboflow Train.**

![Roboflow Wordmark](https://i.imgur.com/dcLNMhV.png)



#Run First

Set up Google Drive connector and pull repo

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
root_dir = "/content/drive/My Drive/"

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive


/content


In [ ]:
!git clone https://github.com/Maadi5/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10668, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10668 (delta 2), reused 0 (delta 0), pack-reused 10662
Receiving objects: 100% (10668/10668), 113.98 MiB | 13.53 MiB/s, done.
Resolving deltas: 100% (7350/7350), done.


In [ ]:
!git clone https://github.com/Maadi5/yolov5
%cd /content/drive/My Drive/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 10671, done.
remote: Total 10671 (delta 0), reused 0 (delta 0), pack-reused 10671
Receiving objects: 100% (10671/10671), 113.92 MiB | 20.67 MiB/s, done.
Resolving deltas: 100% (7360/7360), done.
/content/yolov5


In [3]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 596 kB 5.1 MB/s 
Setup complete. Using torch 1.10.0+cu111 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


## Download Correctly Formatted Custom Dataset 

Downloading the dataset from Roboflow. Use the "**YOLOv5 PyTorch**"

In [4]:
#follow the link below to get your download code from from Roboflow
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")

     |████████████████████████████████| 145 kB 5.0 MB/s 
     |████████████████████████████████| 178 kB 59.0 MB/s 
     |████████████████████████████████| 1.1 MB 63.6 MB/s 
     |████████████████████████████████| 67 kB 7.5 MB/s 
     |████████████████████████████████| 138 kB 75.0 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=roboflow-yolov5


In [ ]:
%cd /content/yolov5


/content/yolov5


In [6]:
rf = Roboflow(api_key="DNKHdcdR92MMLQHABY9a")
project = rf.workspace().project("hardhat-head-yolo")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Hardhat/Head-YoLo-1 in yolov5pytorch::  80%|████████  | 7608/9509 [01:46<00:26, 71.25it/s]


KeyboardInterrupt: ignored

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

# Define Model Configuration and Architecture

Model config and arch

In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat /content/yolov5/models/yolov5s.yaml

In [7]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [8]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

FileNotFoundError: ignored

# Train Custom YOLOv5 Detector


In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 200 --weights yolov5s.pt --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --name yolov5s_results  --cache

/content/yolov5
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 1
Unpacking objects: 100% (3/3), done.
From https://github.com/Maadi5/yolov5
   05581ce..72b54e7  master     -> origin/master
github: ⚠️ WARNING: code is out of date by 1 commit. Use 'git pull' to update or 'git clone https://github.com/Maadi5/yolov5' to download latest.
YOLOv5 v4.0-132-g05581ce torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=True, cfg='./models/custom_yolov5s.yaml', data='/content/yolov5/Hardhat/Head-YoLo-1/data.yaml', device='', entity=None, epochs=200, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='yolov5s_results', noautoanchor=Fals

#Run Inference
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [12]:
%cd /content/drive/My Drive/yolov5


/content/drive/My Drive/yolov5


In [ ]:
#place video_file in root_dir (/yolov5) in google drive
# frames from video will be output in ideo_frames directory
#Target_fps 8 for the video in consideration
!python video2frames.py test_video.mp4 video_frames --maxframes 8000

In [15]:
#run inference on videoframes - store label of predictions (OIDv4 TXT) in labels folder (/pred_frames)


!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source video_frames  --project pred_frames

Streaming output truncated to the last 5000 lines.
image 558/5555 /content/drive/MyDrive/yolov5/out23/frame_0000557.jpg: 256x416 Done. (0.012s)
image 559/5555 /content/drive/MyDrive/yolov5/out23/frame_0000558.jpg: 256x416 Done. (0.013s)
image 560/5555 /content/drive/MyDrive/yolov5/out23/frame_0000559.jpg: 256x416 Done. (0.012s)
image 561/5555 /content/drive/MyDrive/yolov5/out23/frame_0000560.jpg: 256x416 Done. (0.012s)
image 562/5555 /content/drive/MyDrive/yolov5/out23/frame_0000561.jpg: 256x416 Done. (0.012s)
image 563/5555 /content/drive/MyDrive/yolov5/out23/frame_0000562.jpg: 256x416 Done. (0.014s)
image 564/5555 /content/drive/MyDrive/yolov5/out23/frame_0000563.jpg: 256x416 Done. (0.012s)
image 565/5555 /content/drive/MyDrive/yolov5/out23/frame_0000564.jpg: 256x416 Done. (0.011s)
image 566/5555 /content/drive/MyDrive/yolov5/out23/frame_0000565.jpg: 256x416 Done. (0.011s)
image 567/5555 /content/drive/MyDrive/yolov5/out23/frame_0000566.jpg: 256x416 Done. (0.012s)
image 568/5555 /con

In [17]:
!pip install ffmpeg-python
import ffmpeg

(
    ffmpeg
    .input('pred_frames/exp/*.jpg', pattern_type = 'glob', framerate=8)
    .output('out_movie.mp4')
    .run()
)

(None, None)